# Clustering Zooniverse Marks to count Iguanas
The goal is to find the best method to cluster the data and find the best number of clusters.
The benchmark is a gold standard dataset obtained by experts.

In [108]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./")
sys.path.append("./zooniverse")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Intro
### Retrieve a Classification report from Zooniverse
Export the classification export from your zooniverse project.
https://www.zooniverse.org/lab/11905/data-exports

This leads to a csv file which can be used for the analysis which should be renamed to `iguanas-from-above-classifications.csv` and placed in the `input_path` directory.
The methods do not use methods from zooniverse. It is a custom implementation.

An alternative would be to use the [code provided by zooniverse](https://github.com/zooniverse/Data-digging/tree/master/notebooks_ProcessExports)
(Bird Count Example)[https://github.com/zooniverse/Data-digging/blob/master/scripts_ProjectExamples/seabirdwatch/bird_count.py]

This notebooks assumes the data is flat and prepared. An alternative format would be the [cesar aggregation format](https://github.com/zooniverse/aggregation-for-caesar)

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

### Processing workflow

The Data is flatted and filtered
![Image](images/dataprocessing-DataFiltering.png)

In each phase we have the following number of images if they are filtered for at least 4 true marks by users
1. phase 105
2. phase 160
3. phase 86


## Load the data

In [109]:
from zooniverse.utils.data_format import data_prep
from pathlib import Path

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold
from zooniverse.config import get_config

# Phase Selection
phase_tag = "Iguanas 1st launch"
# phase_tag = "Iguanas 2nd launch"
# phase_tag = "Iguanas 3rd launch"

### use either the subset of the subset
use_gold_standard_subset = "expert_goldstandard" # Use the X-T2-GS-results-5th-0s as the basis
# use_gold_standard_subset = "expert" # Use the expert-GS-Xphase as the basis

## Input Path of all the data
input_path = Path("/Users/christian/data/zooniverse")

# Location for the analysis Results
output_path = Path(input_path.joinpath(f"2024_04_22_{use_gold_standard_subset}_analysis").joinpath(phase_tag))

output_path.mkdir(exist_ok=True, parents=True)

reprocess = True # if True, the raw classification data is reprocessed. If False, the data is loaded from disk




debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook

user_threshold = None # None or a number, filter records which have less than these user interactions.

# Location for plots
output_plot_path = output_path.joinpath("plots")
output_plot_path.mkdir(parents=True, exist_ok=True)


## Look into the config
This Config points to all files necessary for the analysis + the result files

In [110]:
config = get_config(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('/Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-1stphase.csv'),
 'gold_standard_image_subset': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/1-T2-GS-results-5th-0s.csv'),
 'image_source': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/1st launch'),
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_22_expert_goldstandard_analysis/Iguanas 1st launch/yes_no_dataset_Iguanas 1st launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_22_expert_goldstandard_analysis/Iguanas 1st launch/flat_dataset_Iguanas 1st launch.csv'),
 'flat_panoptes_points': PosixPath('/Users/christian/data/zooniverse/2024_04_22_expert_goldstandard_analysis/Iguanas 1st launch/flat_panoptes_points_Iguana

In [111]:
from zooniverse.utils.anonymize import UserAnonymizer

if reprocess:
    ds_stats = data_prep(phase_tag=phase_tag, 
                         output_path=output_path, 
                         input_path=input_path,
                         filter_combination=use_gold_standard_subset, 
                         config=config)
    # # Anomymise the data to prevent usernames and user_ids to become public
    # anonymizer = UserAnonymizer(config["flat_dataset"])
    # anonymizer.anonymize_data()
    # anonymizer.save_anonymized_data(config["flat_dataset"])
    # 
    # anonymizer = UserAnonymizer(config["merged_dataset"])
    # anonymizer.anonymize_data()
    # anonymizer.save_anonymized_data(config["merged_dataset"])
    
    print(ds_stats)

2024-04-22 11:12:40.940 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:181 - found a new phase tag: Iguanas 1st launch


{'retired': None, 'link': 'https://www.flickr.com/photos/hellie55/6713782851', 'origin': 'Flickr', 'license': 'Creative Commons - share adapt attribute', 'subject_id': '3', 'attribution': 'hehaden', 'image_name_1': '6713782851_82fc8c73e5_z.jpg', '#secret_description': 'juvenile kittehs practice break-in at the catnip factory'}
{'retired': None, 'link': 'https://www.flickr.com/photos/davebloggs007/15899553966', 'origin': 'Flickr', 'license': 'Creative Commons - share adapt attribute', 'subject_id': '16', 'attribution': 'davebloggs007', 'image_name_1': '15899553966_5debf88369_z.jpg', '#secret_description': 'makeover kitteh not sure these new contacts are sassy enough'}
{'retired': None, 'link': 'https://www.flickr.com/photos/odoketa/8762013777', 'origin': 'Flickr', 'license': 'Creative Commons - share adapt attribute', 'subject_id': '4', 'attribution': 'David Barber', 'image_name_1': '8762013777_1fa7d91e98_z.jpg', '#secret_description': 'hunter kitteh only willing to go so far to capture

2024-04-22 11:12:52.000 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:181 - found a new phase tag: Main Workflow
2024-04-22 11:12:54.048 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:181 - found a new phase tag: survey tool 061417
2024-04-22 11:12:56.358 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:181 - found a new phase tag: Iguanas 2nd launch
2024-04-22 11:13:02.182 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:181 - found a new phase tag: Iguanas 3rd launch
2024-04-22 11:13:23.565 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:181 - found a new phase tag: Iguanas 4th launch
2024-04-22 11:13:23.567 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:181 - found a new phase tag: Plastics GS dataset
2024-04-22 11:13:23.589 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:267 - generating flat dataframe from 

                                       filename  images
0           flat_dataset_Iguanas 1st launch.csv    8260
1                                    1st launch    2737
2                        expert-GS-1stphase.csv    2733
3                    1-T2-GS-results-5th-0s.csv     107
4  flat_dataset_filtered_Iguanas 1st launch.csv     107


In [112]:
if plot_diagrams == False:
    output_plot_path = None

# the flattened, filtered marks from zooniverse.
df_merged_dataset = pd.read_csv(config["merged_dataset"])

# data for reference
df_goldstandard_expert_count = pd.read_csv(config["gold_standard_image_subset"], sep=";")
df_expert_count = pd.read_csv(config["goldstandard_data"], sep=";")

### Optional Debugging

In [113]:
## Debugging helpers
if phase_tag == "Iguanas 1st launch":    
    if debug:
        df_merged_dataset = df_merged_dataset[df_merged_dataset.image_name.isin(["SFM01-2-2-2_333.jpg", "SFM01-2-2-2_334.jpg", "SFM01-2-2-3_201.jpg"])]

elif phase_tag == "Iguanas 2nd launch":
    if debug:
        df_merged_dataset = df_merged_dataset[
           df_merged_dataset.subject_id.isin([72373250])]
           # df_merged_dataset.image_name.isin(["FMO03-1_72.jpg"])]

    
elif phase_tag == "Iguanas 3rd launch":

    # this user is a spammer
    df_merged_dataset = df_merged_dataset[df_merged_dataset.user_id != 2581179]
    
    if debug:
        df_merged_dataset = df_merged_dataset[
           df_merged_dataset.image_name.isin(["FMO03-2_70.jpg", "MBN04-2_182.jpg", "EGI08-2_78.jpg"])]
            
    


## Look at the data


In [114]:
## Look at the data
df_merged_dataset.drop("user_name", axis=1)
df_merged_dataset.user_name.unique()

df_merged_dataset.groupby('user_name').size()

user_name
18asewell                2
352669                   7
700yapph                 1
7wonders                13
A.Over                   2
                        ..
williaml96               1
wolfhaley                3
xavierm_at_microsoft     2
zmaeapril                1
zsweeney94               3
Length: 558, dtype: int64

### Filter User if necessary and Marks


In [115]:
print(f"Before filtering: {df_merged_dataset.subject_id.nunique()}")
# There images in which some people said there are iguanas, but then didn't mark them. Clustering with fewer than 3 dots doesn't make sense
if user_threshold is not None:
    print(f"filtering records which have less than {user_threshold} interactions.")
    df_merged_dataset = filter_df_user_threshold(df_merged_dataset, user_threshold=user_threshold)
    
    
from zooniverse.utils.filters import filter_remove_marks
# Check if partials are still in the data. There shouldn't be any
df_merged_dataset = filter_remove_marks(df_merged_dataset)


2024-04-22 11:13:27.309 | INFO     | zooniverse.utils.filters:filter_remove_marks:36 - before filter func: 2630
2024-04-22 11:13:27.310 | WARNING  | zooniverse.utils.filters:filter_remove_marks:41 - removed 0 partial marks
2024-04-22 11:13:27.311 | WARNING  | zooniverse.utils.filters:filter_remove_marks:42 - removed 2630 marks
2024-04-22 11:13:27.311 | WARNING  | zooniverse.utils.filters:filter_remove_marks:43 - After filter_func 107 images are left


Before filtering: 107


### Are there anonymous users in the data?
There should be

In [116]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

,Unnamed: 0,flight_site_code,workflow_id,workflow_version,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name,mission_name,image_path,width,height
13,13,SFM1,14370,134.236,SFM01-1-2_59.jpg,47968506,116.633331,153.966675,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,NaN,not-logged-in-33bc5b21c88460dbfea6,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
15,15,SFM1,14370,134.236,SFM01-1-1_184.jpg,47968013,279.644714,199.890869,Adult Male alone,Iguanas 1st launch,NaN,not-logged-in-33bc5b21c88460dbfea6,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,600,630
16,16,SFM1,14370,134.236,SFM01-1-1_184.jpg,47968013,407.698547,321.648682,Adult Male alone,Iguanas 1st launch,NaN,not-logged-in-33bc5b21c88460dbfea6,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,600,630
21,21,SFM1,14370,134.236,SFM01-2-2-2_302.jpg,47969853,549.633301,540.966675,Adult Male alone,Iguanas 1st launch,NaN,not-logged-in-33bc5b21c88460dbfea6,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,667,649
24,24,SFM1,14370,134.236,SFM01-2-2-3_149.jpg,47970139,57.739017,178.533081,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,NaN,not-logged-in-33bc5b21c88460dbfea6,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2585,2585,SFM1,14370,134.236,SFM01-1-2_284.jpg,47968423,18.856710,494.673126,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,NaN,not-logged-in-93e07d6b2a99b8725f21,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
2586,2586,SFM1,14370,134.236,SFM01-1-2_284.jpg,47968423,94.404442,522.908142,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,NaN,not-logged-in-93e07d6b2a99b8725f21,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
2587,2587,SFM1,14370,134.236,SFM01-1-2_284.jpg,47968423,162.702652,474.069214,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,NaN,not-logged-in-93e07d6b2a99b8725f21,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
2588,2588,SFM1,14370,134.236,SFM01-1-2_284.jpg,47968423,85.247139,474.450745,Adult Male in a lek,Iguanas 1st launch,NaN,not-logged-in-93e07d6b2a99b8725f21,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630


In [117]:
# Amount of images
df_merged_dataset["subject_id"].nunique()

107

In [118]:
## After filtering there
df_merged_dataset

,Unnamed: 0,flight_site_code,workflow_id,workflow_version,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name,mission_name,image_path,width,height
0,0,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,376.160156,84.558594,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
1,1,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,272.695312,297.894531,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
2,2,SFM1,14370,134.236,SFM01-2-2-3_137.jpg,47970105,111.898438,410.011719,Adult Male alone,Iguanas 1st launch,2068663.0,aymor,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
3,3,SFM1,14370,134.236,SFM01-1-2_264.jpg,47968406,462.800781,390.786377,Adult Male in a lek,Iguanas 1st launch,1878692.0,drewavery,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
4,4,SFM1,14370,134.236,SFM01-1-2_264.jpg,47968406,545.598145,423.652527,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,1878692.0,drewavery,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,2625,SFM1,14370,134.236,SFM01-2-2-3_194.jpg,47970222,64.312500,166.000000,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,2113479.0,axiongxx,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,589,573
2626,2626,SFM1,14370,134.236,SFM01-2-2-1_120.jpg,47969236,231.432205,154.631561,Adult Male alone,Iguanas 1st launch,NaN,not-logged-in-341c84c0e52bdc34f8f8,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,666,648
2627,2627,SFM1,14370,134.236,SFM01-1-2_210.jpg,47968359,316.314850,392.354614,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,2136122.0,H.axson,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630
2628,2628,SFM1,14370,134.236,SFM01-1-2_210.jpg,47968359,326.542725,402.156372,"Others (females, young males, juveniles and ta...",Iguanas 1st launch,2136122.0,H.axson,SFM1,/Users/christian/data/zooniverse/Images/Zooniv...,599,630


In [119]:
# how many marks per user
df_merged_dataset[["user_id", "x"]].groupby("user_id").count().head()

,x
user_id,
1476.0,1
2842.0,14
109942.0,1
114901.0,4
120287.0,2


### Gold standard data
For reference

In [120]:
df_goldstandard_expert_count["subject_id"].nunique()

107

In [121]:
# look at the
df_goldstandard_expert_count.count()

subject_id    107
Median0s      107
Mean0s        107
Max0s         107
Std0s         102
Median.r      107
Mean.r        107
Mode0s        107
dtype: int64

In [122]:
# How many images are left in the zooniverse dataset?
len(list(df_merged_dataset.image_name.unique()))

107

In [123]:
#Is there an image in the goldstandard, which is not in the classifcations?
print(f"images in df_goldstandard_expert_count but not in df_merged_dataset: {len(set(df_goldstandard_expert_count.subject_id) - set(df_merged_dataset.subject_id.unique()))}")


images in df_goldstandard_expert_count but not in df_merged_dataset: 0


In [124]:
df_goldstandard_expert_count.subject_id

0      47967876
1      47967959
2      47967961
3      47967975
4      47968013
         ...   
102    48026608
103    48026629
104    48026645
105    48026817
106    48027975
Name: subject_id, Length: 107, dtype: int64

In [125]:
df_expert_count.subject_id


0       47969795
1       47969531
2       47969760
3       47969815
4       47969821
          ...   
2728    47990261
2729    47990268
2730    47990280
2731    47990285
2732    47990290
Name: subject_id, Length: 2733, dtype: int64

In [126]:
df_expert_count.count_total.sum()

422

In [127]:
# How many images are in the filtered flat zooniverse dataset
df_merged_dataset["subject_id"].nunique()

107

In [128]:
## plot some of the marks
from zooniverse.utils.plotting import plot_zooniverse_user_marks_v2

if phase_tag in["Iguanas 1st launch", "Iguanas 2nd launch"]  and  ( plot_diagrams or show_plots ) :
    for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
        
        ## plot the marks
        markers_plot_path = plot_zooniverse_user_marks_v2(df_image_name,
                                                          image_path=df_image_name.iloc[0]["image_path"],
                                                          image_name=image_name,
                                                          output_path=output_plot_path, show=show_plots, title=f"Markers for {image_name}", fig_size=(5,5))
        

## Clustering

### Basic Statics like mean, median, mode

In [129]:
from sklearn.metrics import mean_squared_error
from zooniverse.analysis import get_mark_overview

basic_stats = []
kmeans_knee_stats = []
kmeans_silouettes = []
mse_errors = {}


for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)

    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"],
                                                        subject_id=df_image_name.iloc[0]["subject_id"])

    ### basic statistics like mean, median
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    

## join the gold standard data to the basic stats
if use_gold_standard_subset is not None:
    df_comparison = df_expert_count.merge(df_basic_stats, on='image_name', how='left')
else:
    df_comparison = df_basic_stats


In [130]:
df_basic_stats

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
0,SFM01-1-1_114.jpg,47967876,1.0,1.11,1,1,[1],1.0,9,10,"[1, 1, 1, 1, 1, 1, 1, 1, 2]"
1,SFM01-1-1_154.jpg,47967959,1.0,1.00,1,1,[1],1.0,11,11,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,SFM01-1-1_155.jpg,47967961,5.0,5.25,4,4,[4],4.0,4,21,"[4, 4, 6, 7]"
3,SFM01-1-1_163.jpg,47967975,1.0,1.00,1,1,[1],1.0,6,6,"[1, 1, 1, 1, 1, 1]"
4,SFM01-1-1_184.jpg,47968013,1.0,1.20,1,1,[1],1.0,5,6,"[1, 1, 1, 1, 2]"
...,...,...,...,...,...,...,...,...,...,...,...
102,SRL01-2-2_54.jpg,48026608,4.0,3.63,4,4,[4],4.0,19,69,"[2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
103,SRL01-2-2_55.jpg,48026629,1.0,1.00,1,1,[1],1.0,17,17,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
104,SRL01-2-2_62.jpg,48026645,2.0,1.73,2,2,[2],2.0,15,26,"[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 4]"
105,SRL02-1_33.jpg,48026817,1.0,1.09,1,1,[1],1.0,11,12,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]"


In [131]:
df_basic_stats["mode_count"].sum()

[1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 13,
 1,
 1,
 1,
 2,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 9,
 1,
 1,
 1,
 1,
 1,
 4,
 10,
 1,
 3,
 1,
 2,
 3,
 1,
 8,
 12,
 1,
 1,
 1,
 1,
 2,
 1,
 4,
 11,
 2,
 1,
 1,
 2,
 2,
 9,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 6,
 7,
 8,
 1,
 2,
 3,
 1,
 7,
 10,
 2,
 1,
 1,
 4,
 1,
 5,
 7,
 10,
 1,
 1,
 2,
 9,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 2,
 1,
 2]

In [132]:
# There might be records with too few annotations
df_comparison[(df_comparison.count_total > 0) & (df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

,subspecies,island,site_name,subject_group,image_name,subject_id_x,presence_absence,count_male-lek,count_male-no-lek,count_others,...,subject_id_y,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
718,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_350.jpg,47969975,Y,0,1,0,...,47969975.0,1.0,1.0,1.0,1.0,[1],1.0,4.0,4.0,"[1, 1, 1, 1]"
732,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-3_111.jpg,47970029,Y,0,0,1,...,47970029.0,1.0,1.0,1.0,1.0,[1],1.0,4.0,4.0,"[1, 1, 1, 1]"
666,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_278.jpg,47969782,Y,0,0,1,...,47969782.0,2.0,2.0,1.0,3.0,"[1, 3]",2.0,2.0,4.0,"[1, 3]"
878,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-extra_114.jpg,47970403,Y,0,1,0,...,47970403.0,1.0,1.0,1.0,1.0,[1],1.0,2.0,2.0,"[1, 1]"
743,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-3_123.jpg,47970063,Y,0,0,1,...,47970063.0,2.0,2.0,2.0,2.0,[2],2.0,1.0,2.0,[2]


In [133]:
# images with an expert count of more than 0 and less than 5 different users
df_comparison[(df_comparison.count_total > 0) & (df_comparison.users < 5)].sort_values(by="users", ascending=False)


,subspecies,island,site_name,subject_group,image_name,subject_id_x,presence_absence,count_male-lek,count_male-no-lek,count_others,...,subject_id_y,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
61,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-1-1_155.jpg,47967961,Y,1,0,3,...,47967961.0,5.0,5.25,4.0,4.0,[4],4.000000,4.0,21.0,"[4, 4, 6, 7]"
236,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-1-2_210.jpg,47968359,Y,0,0,1,...,47968359.0,1.0,1.25,1.0,1.0,[1],1.000000,4.0,5.0,"[1, 1, 1, 2]"
590,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-1_321.jpg,47969518,Y,0,1,0,...,47969518.0,1.0,1.25,1.0,1.0,[1],1.000000,4.0,5.0,"[1, 1, 1, 2]"
708,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_331.jpg,47969939,Y,0,1,2,...,47969939.0,2.0,2.00,2.0,2.0,[2],2.000000,4.0,8.0,"[1, 2, 2, 3]"
718,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_350.jpg,47969975,Y,0,1,0,...,47969975.0,1.0,1.00,1.0,1.0,[1],1.000000,4.0,4.0,"[1, 1, 1, 1]"
732,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-3_111.jpg,47970029,Y,0,0,1,...,47970029.0,1.0,1.00,1.0,1.0,[1],1.000000,4.0,4.0,"[1, 1, 1, 1]"
767,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-3_152.jpg,47970148,Y,1,1,6,...,47970148.0,6.5,5.50,1.0,8.0,"[1, 6, 7, 8]",5.500000,4.0,22.0,"[1, 6, 7, 8]"
448,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-1_305.jpg,47969049,Y,0,1,1,...,47969049.0,2.0,2.33,1.0,4.0,"[1, 2, 4]",2.333333,3.0,7.0,"[1, 2, 4]"
0,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,Y,0,2,0,...,47969795.0,2.5,2.50,2.0,3.0,"[2, 3]",2.500000,2.0,5.0,"[2, 3]"
577,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-1_293.jpg,47969478,N,0,0,9,...,47969478.0,7.5,7.50,6.0,9.0,"[6, 9]",7.500000,2.0,15.0,"[6, 9]"


In [134]:
df_comparison["count_total"].sum()

422

### Fill NaN values with 0 because the errors can't be calculated otherwise

In [135]:

## Fill NaN values with 0 because the errors can't be calculated otherwise
df_comparison.fillna(0, inplace=True)


In [136]:

mse_errors["median_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.median_count,
                                                     squared=False)
mse_errors["mean_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.mean_count, squared=False)

pd.Series(mse_errors)

median_count_rmse    0.626731
mean_count_rmse      0.584600
dtype: float64

In [137]:
df_comparison

,subspecies,island,site_name,subject_group,image_name,subject_id_x,presence_absence,count_male-lek,count_male-no-lek,count_others,...,subject_id_y,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
0,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,Y,0,2,0,...,47969795.0,2.5,2.50,2.0,3.0,"[2, 3]",2.5,2.0,5.0,"[2, 3]"
1,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,Y,0,2,2,...,47969531.0,1.0,1.44,1.0,1.0,[1],1.0,9.0,13.0,"[1, 1, 1, 1, 1, 1, 2, 2, 3]"
2,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_270.jpg,47969760,Y,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
3,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_289.jpg,47969815,Y,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
4,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_291.jpg,47969821,Y,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_95.jpg,47990261,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
2729,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_96.jpg,47990268,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
2730,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_97.jpg,47990280,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
2731,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_98.jpg,47990285,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0


In [138]:

pd.Series(mse_errors)

median_count_rmse    0.626731
mean_count_rmse      0.584600
dtype: float64

In [139]:
df_comparison

,subspecies,island,site_name,subject_group,image_name,subject_id_x,presence_absence,count_male-lek,count_male-no-lek,count_others,...,subject_id_y,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
0,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,Y,0,2,0,...,47969795.0,2.5,2.50,2.0,3.0,"[2, 3]",2.5,2.0,5.0,"[2, 3]"
1,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,Y,0,2,2,...,47969531.0,1.0,1.44,1.0,1.0,[1],1.0,9.0,13.0,"[1, 1, 1, 1, 1, 1, 2, 2, 3]"
2,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_270.jpg,47969760,Y,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
3,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_289.jpg,47969815,Y,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
4,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_291.jpg,47969821,Y,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_95.jpg,47990261,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
2729,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_96.jpg,47990268,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
2730,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_97.jpg,47990280,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0
2731,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_98.jpg,47990285,N,0,0,0,...,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0


In [140]:
df_comparison[["median_count", "mean_count", "count_total"]].sum()

median_count    236.00
mean_count      256.87
count_total     422.00
dtype: float64

### DBSCAN clustering and take the variant with the best silouette score for each image


In [141]:
## fixes the problem with the silouette score sorting
from zooniverse.analysis import compare_dbscan

eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_variants = [3, 5, 8, 10]
if debug:
    eps_variants = [0.3]
    min_samples_variants = [3]
params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]

db_scan_results = {}
db_scan_best_results = []
db_scan_best_bic_results = []
for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    if df_image_name.shape[0] >= 5:
        dbscan_localization = compare_dbscan(
            # phase_tag=phase_tag,
            params=params,
            df_flat=df_image_name,
            # output_path=output_path,
            output_plot_path=output_plot_path,
            plot=show_plots,
    
        )
    
        db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
    
        # DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
        # Sillouette Scoring needs a minimum of 2 clusters
        # if there are points in decent radius they will belong to a cluster
        if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
            db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
            db_scan_best_bic_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
            # If two or more cluster seem to exists take ones with the best Silouette score
        else:  
            # take the best result by silouette score if there are more clusters then 1
            db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values(["dbscan_silouette_score", "dbscan_count"], ascending=[False, False]).iloc[0])

df_dbscan_localization = pd.concat([*db_scan_results.values()])
df_scan_best_results = pd.DataFrame(db_scan_best_results)



/var/folders/2k/78nn7s4548986wsjh29rhj9w0000gn/T/ipykernel_97849/4033969769.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dbscan_localization = pd.concat([*db_scan_results.values()])


In [142]:
df_scan_best_results

,image_name,subject_id,dbscan_count,dbscan_noise,dbscan_silouette_score,dbscan_BIC_score,eps,min_samples
16,SFM01-1-1_114.jpg,47967876,1,2,NaN,NaN,0.30,3
24,SFM01-1-1_154.jpg,47967959,1,8,NaN,NaN,0.50,3
24,SFM01-1-1_155.jpg,47967961,3,4,0.465019,NaN,0.50,3
24,SFM01-1-1_163.jpg,47967975,1,3,NaN,NaN,0.50,3
16,SFM01-1-1_184.jpg,47968013,1,1,NaN,NaN,0.30,3
...,...,...,...,...,...,...,...,...
24,SRL01-2-2_54.jpg,48026608,4,1,0.790920,NaN,0.50,3
24,SRL01-2-2_55.jpg,48026629,2,7,0.123768,NaN,0.50,3
20,SRL01-2-2_62.jpg,48026645,2,3,0.712219,NaN,0.40,3
0,SRL02-1_33.jpg,48026817,0,12,NaN,NaN,0.01,3


Here it can be seen why the silouette score is difficult because it is often undefined.

In [143]:
## save the combinations of parameters, which maximized the silouette score.

df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])
df_scan_best_results

,image_name,subject_id,dbscan_count,dbscan_noise,dbscan_silouette_score,dbscan_BIC_score,eps,min_samples
16,SFM01-1-1_114.jpg,47967876,1,2,NaN,NaN,0.30,3
24,SFM01-1-1_154.jpg,47967959,1,8,NaN,NaN,0.50,3
24,SFM01-1-1_155.jpg,47967961,3,4,0.465019,NaN,0.50,3
24,SFM01-1-1_163.jpg,47967975,1,3,NaN,NaN,0.50,3
16,SFM01-1-1_184.jpg,47968013,1,1,NaN,NaN,0.30,3
...,...,...,...,...,...,...,...,...
24,SRL01-2-2_54.jpg,48026608,4,1,0.790920,NaN,0.50,3
24,SRL01-2-2_55.jpg,48026629,2,7,0.123768,NaN,0.50,3
20,SRL01-2-2_62.jpg,48026645,2,3,0.712219,NaN,0.40,3
0,SRL02-1_33.jpg,48026817,0,12,NaN,NaN,0.01,3


In [144]:
df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil" }, inplace=True)

df_comparison = df_comparison.merge(df_scan_best_results, on='image_name', how='left')

In [145]:
df_comparison.fillna(0, inplace=True)

mse_errors["dbscan_count_sil_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.dbscan_count_sil, squared=False)

pd.Series(mse_errors)

median_count_rmse        0.626731
mean_count_rmse          0.584600
dbscan_count_sil_rmse    0.665385
dtype: float64

In [146]:

df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

,subspecies,island,site_name,subject_group,image_name,subject_id_x,presence_absence,count_male-lek,count_male-no-lek,count_others,...,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count,subject_id,dbscan_count_sil
0,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,Y,0,2,0,...,2.50,2.0,3.0,"[2, 3]",2.5,2.0,5.0,"[2, 3]",47969795.0,0.0
1,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,Y,0,2,2,...,1.44,1.0,1.0,[1],1.0,9.0,13.0,"[1, 1, 1, 1, 1, 1, 2, 2, 3]",47969531.0,2.0
2,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_270.jpg,47969760,Y,0,0,0,...,0.00,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
3,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_289.jpg,47969815,Y,0,0,0,...,0.00,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
4,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_291.jpg,47969821,Y,0,0,0,...,0.00,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_95.jpg,47990261,N,0,0,0,...,0.00,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
2729,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_96.jpg,47990268,N,0,0,0,...,0.00,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
2730,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_97.jpg,47990280,N,0,0,0,...,0.00,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
2731,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_98.jpg,47990285,N,0,0,0,...,0.00,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0


### HDBSCAN clustering for each image

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html#sklearn.cluster.HDBSCAN states: "A distance threshold. Clusters below this value will be merged."



In [147]:
from zooniverse.analysis import hdbscan

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"],
                                                        subject_id=df_image_name.iloc[0]["subject_id"])
    
    # if less than min_cluster_sizes points are available clustering makes no sense
    if df_image_name.shape[0] >= 5: # If num_samples is 5 for the min_cluster_size is 5 there is no point in passing data with less than 5 samples
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = hdbscan(df_marks=df_image_name[["x", "y"]],
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     subject_id=df_image_name.iloc[0]["subject_id"],
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



In [148]:
df_hdbscan.drop(["with_noise"], axis=1, inplace=True)
df_hdbscan

,image_name,subject_id,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,47967876,47967876,1,0.0,5,None,5
0,47967959,47967959,1,0.0,5,None,6
0,47967961,47967961,1,0.0,5,None,16
0,47967975,47967975,1,0.0,5,None,1
0,47968013,47968013,1,0.0,5,None,1
...,...,...,...,...,...,...,...
0,48026608,48026608,4,0.0,5,None,1
0,48026629,48026629,1,0.0,5,None,12
0,48026645,48026645,2,0.0,5,None,1
0,48026817,48026817,1,0.0,5,None,7


In [149]:
df_comparison = df_comparison.merge(df_hdbscan, on='subject_id', how='left')
df_comparison.fillna(0, inplace=True)
df_comparison

,subspecies,island,site_name,subject_group,image_name_x,subject_id_x,presence_absence,count_male-lek,count_male-no-lek,count_others,...,sum_annotations_count,annotations_count,subject_id,dbscan_count_sil,image_name_y,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,Y,0,2,0,...,5.0,"[2, 3]",47969795.0,0.0,47969795.0,1.0,0.0,5.0,0,0.0
1,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,Y,0,2,2,...,13.0,"[1, 1, 1, 1, 1, 1, 2, 2, 3]",47969531.0,2.0,47969531.0,1.0,0.0,5.0,0,8.0
2,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_270.jpg,47969760,Y,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_289.jpg,47969815,Y,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_291.jpg,47969821,Y,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_95.jpg,47990261,N,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2729,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_96.jpg,47990268,N,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2730,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_97.jpg,47990280,N,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2731,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_98.jpg,47990285,N,0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [150]:
df_comparison.count_total.fillna(0, inplace=True)
df_comparison.HDBSCAN_count.fillna(0, inplace=True)

df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

mse_errors["hdbscan_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.HDBSCAN_count, squared=False)


saved /Users/christian/data/zooniverse/2024_04_22_expert_goldstandard_analysis/Iguanas 1st launch/Iguanas 1st launch_method_comparison.csv


# A look into the results
Root Means Squared Error for the different methods

In [151]:
df_rmse = pd.DataFrame(pd.Series(mse_errors).sort_values())
df_rmse.to_csv(config["rmse_errors"])


## The sum of the clustering
What is the sum of the methods

In [152]:
df_comparison.subject_id.nunique()

98

In [153]:

df_comparison_sum = df_comparison[["count_total", "median_count", "mean_count", "dbscan_count_sil", "HDBSCAN_count"]].sum().sort_values()
df_comparison_sum.to_csv(config["method_sums"])


In [154]:
print(f"phase_tag: {phase_tag}, user_threshold: {user_threshold}")

phase_tag: Iguanas 1st launch, user_threshold: None


## Compare the numbers
The counts are only for images which were in the dataset after filtering.

### Sum of all the Methods

In [155]:
print(f"{config['method_sums'].name}")
pd.read_csv(config["method_sums"])

Iguanas 1st launch_method_sums.csv


,Unnamed: 0,0
0,dbscan_count_sil,218.00
1,HDBSCAN_count,219.00
2,median_count,236.00
3,mean_count,256.87
4,count_total,422.00


### Root Mean Squared Error

In [156]:
print(f"{config['rmse_errors'].name}")
pd.read_csv(config["rmse_errors"])

Iguanas 1st launch_rmse_errors.csv


,Unnamed: 0,0
0,mean_count_rmse,0.584600
1,hdbscan_count_rmse,0.600341
2,median_count_rmse,0.626731
3,dbscan_count_sil_rmse,0.665385


### Comparison per Image Level

In [157]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])

load /Users/christian/data/zooniverse/2024_04_22_expert_goldstandard_analysis/Iguanas 1st launch/Iguanas 1st launch_method_comparison.csv


,Unnamed: 0,subspecies,island,site_name,subject_group,image_name_x,subject_id_x,presence_absence,count_male-lek,count_male-no-lek,...,sum_annotations_count,annotations_count,subject_id,dbscan_count_sil,image_name_y,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,0,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,Y,0,2,...,5.0,"[2, 3]",47969795.0,0.0,47969795.0,1.0,0.0,5.0,0,0.0
1,1,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,Y,0,2,...,13.0,"[1, 1, 1, 1, 1, 1, 2, 2, 3]",47969531.0,2.0,47969531.0,1.0,0.0,5.0,0,8.0
2,2,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_270.jpg,47969760,Y,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,3,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_289.jpg,47969815,Y,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,4,A. c. trillmichi,Santa Fe,El Miedo,SFM1,SFM01-2-2-2_291.jpg,47969821,Y,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,2728,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_95.jpg,47990261,N,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2729,2729,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_96.jpg,47990268,N,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2730,2730,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_97.jpg,47990280,N,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2731,2731,A. c. godzilla,San Cristobal,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_98.jpg,47990285,N,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


## Discussion:
Clustering works, it yields better numbers than just taking mode,median or mean annotations from the volunteers, because it takes the spatial location of the marker dots into consideration.



Assert the numbers haven't changed

In [158]:
df_comparison_sum

dbscan_count_sil    218.00
HDBSCAN_count       219.00
median_count        236.00
mean_count          256.87
count_total         422.00
dtype: float64

In [159]:
df_comparison.subject_id.nunique()

98

In [160]:
# these are the numbers before the sorting was repaired
if phase_tag == "Iguanas 1st launch" and not debug:
   
    if user_threshold == 3:
        assert df_comparison_sum["median_count"] == 228.5
        assert df_comparison_sum["HDBSCAN_count"] == 244
        assert df_comparison_sum["count_total"] == 422
        
        assert df_comparison.subject_id.nunique() == 2733

if phase_tag == "Iguanas 2nd launch" and not debug:

    if user_threshold == 3:
        assert df_comparison_sum["median_count"] == 475
        assert df_comparison_sum["HDBSCAN_count"] == 541.0
        assert df_comparison_sum["count_total"] == 600
        
        assert df_comparison.subject_id.nunique() == 456

if phase_tag == "Iguanas 3rd launch" and not debug:
        
    if user_threshold == 3:
        assert df_comparison_sum["median_count"] == 314.5
        assert df_comparison_sum["HDBSCAN_count"] == 358
        assert df_comparison_sum["count_total"] == 388
        
        assert df_comparison.subject_id.nunique() == 1156
